<a href="https://colab.research.google.com/github/syakesaba/jupyter-notebooks/blob/main/langfuse_pydantic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -qq pydantic_ai[logfire] nest_asyncio
import os
import base64
import nest_asyncio
nest_asyncio.apply() # Google Colab自体がasyncio配下で動いているのでネストさせる。
import logfire
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY") # Secrets（🔑）からGOOGLE_API_KEYをNotebook access可能にする
LANGFUSE_PUBLIC_KEY = userdata.get("LANGFUSE_PUBLIC_KEY") # Secrets（🔑）からLANGFUSE_PUBLIC_KEYをNotebook access可能にする
LANGFUSE_SECRET_KEY = userdata.get("LANGFUSE_SECRET_KEY") # Secrets（🔑）からLANGFUSE_SECRET_KEYをNotebook access可能にする
LANGFUSE_HOST = userdata.get("LANGFUSE_HOST") # Secrets（🔑）からLANGFUSE_HOSTをNotebook access可能にする

# GeminiModelに関する環境変数をGoogle Colabに作成
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY

# OTLP_EXPORTERに関する環境変数をGoogle Colabに作成
# https://langfuse.com/docs/integrations/pydantic-ai
LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = f"{LANGFUSE_HOST}/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# logfire cloudへの余計な送信を無効化
import logfire
_ = logfire.configure(
    service_name='MyGoogleColab',
    # Sending to Logfire is on by default regardless of the OTEL env vars.
    send_to_logfire=False,
)

In [20]:
from pydantic_ai import Agent
from pydantic_ai.models.gemini import GeminiModel
from pydantic import BaseModel
model = GeminiModel("gemini-2.0-flash")
agent = Agent(model=model, instrument=True)
result = await agent.run("1+1=?")
print(result.data)

14:45:30.155 agent run
14:45:30.162   preparing model request params
14:45:30.162   chat gemini-2.0-flash
1 + 1 = 2



In [24]:
import random
from typing import Annotated, Optional
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.gemini import GeminiModel
from pydantic import BaseModel, StringConstraints
from dataclasses import dataclass

model = GeminiModel("gemini-2.0-flash")

@dataclass
class JankenResult:
    result: Annotated[str, StringConstraints(pattern="^(グー|チョキ|パー)$")] | None

@dataclass
class JankenResults:
    mario: JankenResult
    luigi: JankenResult

mario = Agent(
    model=model, name="Mario",
    system_prompt="あなたはジャンケンをするAIエージェントです。"
                  "'get_janken' toolを使用し、ジャンケンで「グー」、「チョキ」、「パー」のどれかをランダムに出します。",
    deps_type=None, result_type=JankenResult, instrument=True
)
luigi = Agent(
    model=model, name="Luigi",
    system_prompt="あなたはジャンケンをするAIエージェントです。"
                  "'get_janken' toolを使用し、ジャンケンで「グー」、「チョキ」、「パー」のどれかをランダムに出します。",
    deps_type=None, result_type=JankenResult, instrument=True
)
peach = Agent(
    model=model, name="Peach",
    system_prompt="あなたはジャンケン勝負の結果を出力するAIエージェントです。"
                  "'prompt_janken' toolを使用し、ジャンケン勝負の結果と勝敗を出力してください。",
    deps_type=JankenResults, result_type=str, instrument=True
)

yoshi = Agent(
    model=model, name="Yoshi",
    system_prompt="あなたは親切なAIエージェントです。"
                  "get_janken_results' toolを使用し結果を教えてください。",
    deps_type=None, result_type=str, instrument=True
)

@mario.tool
def get_janken(ctx: RunContext[None]) -> str:
    return random.choice(["グー", "チョキ", "パー"])

@luigi.tool
def get_janken(ctx: RunContext[None]) -> str:
    return random.choice(["グー", "チョキ", "パー"])

@peach.tool
async def prompt_janken(ctx: RunContext[None]) -> str:
    mario_response = await mario.run(
        f'ジャンケン、',
        usage=ctx.usage,
    )
    luigi_response = await luigi.run(
        f'ジャンケン、',
        usage=ctx.usage,
    )
    jankenResults = JankenResults(mario=mario_response.data, luigi=luigi_response.data)
    print(jankenResults)
    if jankenResults.mario.result == "グー" and jankenResults.luigi.result == "グー":
        return "双方グーで引き分けです。"
    elif jankenResults.mario.result == "グー" and jankenResults.luigi.result == "チョキ":
        return "Marioがグーで勝ちです。"
    elif jankenResults.mario.result == "グー" and jankenResults.luigi.result == "パー":
        return "Luigiがパーで勝ちです。"
    elif jankenResults.mario.result == "チョキ" and jankenResults.luigi.result == "グー":
        return "Luigiがグーで勝ちです。"
    elif jankenResults.mario.result == "チョキ" and jankenResults.luigi.result == "チョキ":
        return "双方チョキで引き分けです。"
    elif jankenResults.mario.result == "チョキ" and jankenResults.luigi.result == "パー":
        return "Marioがチョキで勝ちです。"
    elif jankenResults.mario.result == "パー" and jankenResults.luigi.result == "グー":
        return "Marioがパーで勝ちです。"
    elif jankenResults.mario.result == "パー" and jankenResults.luigi.result == "チョキ":
        return "Luigiがチョキで勝ちです。"
    elif jankenResults.mario.result == "パー" and jankenResults.luigi.result == "パー":
        return "双方パーで引き分けです。"
    else:
        return "不明な結果です。"

@yoshi.tool
async def get_janken_results(ctx: RunContext[None]) -> str:
    response = await peach.run("ジャンケン勝負の結果と勝敗を出力してください", usage=ctx.usage)
    return response.data

result = await yoshi.run("どちらがどの手を出したかと、勝敗を教えて下さい。", deps=None, result_type=str)
print(result.data) # マリオがチョキを出して勝ちました。


14:51:56.989 Yoshi run
14:51:56.990   preparing model request params
14:51:56.991   chat gemini-2.0-flash
14:51:57.898   running tools: get_janken_results
14:51:57.899     Peach run
14:51:57.900       preparing model request params
14:51:57.900       chat gemini-2.0-flash
14:51:58.369       running tools: prompt_janken
14:51:58.371         Mario run
14:51:58.371           preparing model request params
14:51:58.372           chat gemini-2.0-flash
14:51:58.866           running tools: get_janken
14:51:58.868           preparing model request params
14:51:58.869           chat gemini-2.0-flash
14:51:59.395         Luigi run
14:51:59.395           preparing model request params
14:51:59.396           chat gemini-2.0-flash
14:52:00.155           running tools: get_janken
14:52:00.157           preparing model request params
14:52:00.157           chat gemini-2.0-flash
JankenResults(mario=JankenResult(result='チョキ'), luigi=JankenResult(result='パー'))
14:52:00.627       preparing model request